In [48]:
from run import ms

In [59]:
import pandas as pd
df = pd.DataFrame({"ids": ms.ids, "jobs": ms.jobs, "job_types": ms.job_types})

In [ ]:
meng_xin = [Nemo尼莫, Hoyt, 枇杷树, 点恋线, 铅笔笔拳, timemei, 强人锁男, 冰峡江月, 声微饭否, 猫猫爱金币, 朗姆柠梨, 小颖宝, 青哦苹果, 千万恶霸, 义气丶奶]
help_meng_xin = [怪力, 滴滴叭叭, Flash936, 白菜飞]

In [70]:
df.groupby("jobs")[["ids", "job_types"]].agg(list)

,ids,job_types
jobs,,
冰雷,"[天天酱, 冰封de记忆]","[冰雷, 冰雷]"
刀,"[滴滴叭叭, 无敌铁锅, Cris, 暗影灬, 白菜飞, Flash936, 进击的大腿]","[近战, 近战, 近战, 近战, 近战, 近战, 近战]"
圣骑,[日落沙滩前],[近战]
奶,"[千万恶霸, 义气丶奶, 小颖宝, 青哦苹果, 小手很烫]","[奶, 奶, 奶, 奶, 奶]"
弓,"[Nemo尼莫, Hoyt, 枇杷树, 羽寒, 怪力, 越前小憋, 鱼儿爱看雪, 亚太首席技师]","[眼, 眼, 眼, 眼, 眼, 眼, 眼, 眼]"
弩,"[点恋线, 桥本奈奈未]","[眼, 眼]"
拳,"[阿树, 铅笔笔拳]","[拳, 拳]"
标,"[timemei, 喃]","[远程, 远程]"
火,"[强人锁男, 冰峡江月, Lancer, 声微饭否, 情人游天地, 一天也]","[火, 火, 火, 火, 火, 火]"


In [ ]:
ms = MappingSource(
    ids=id,                # 你的主数据 id 列表
    jobs=job,              # 可不使用但可保留
    job_types=job_type,    # 与 id 对齐的职业类型
)

report = run("temp.csv", ms, random_seed=200)

# 成功分组
print(report.grouped)        # 各队成员ID
print(report.grouped_jobs)   # 各队成员职业

# 问题汇总
print("Warnings:", report.warnings)
print("Errors:", report.errors)
print("Unmapped:", report.unmapped_ids)
print("Invalid:", report.invalid_ids)
print("Leftover:", report.leftover_buckets)

for t_i, (id_i, job_i) in enumerate(zip(report.grouped, report.grouped_jobs)):
    print(f"{t_i+1}队")
    print(f"{id_i}")
    print(f"{job_i}")
    print(f"==========")

[['千万恶霸', '阿树', '声微饭否', 'Flash936', '厉飞羽', '进击的大腿'], ['铅笔笔拳', '桥本奈奈未', '喃', 'timemei', '怪力', 'Nemo尼莫'], ['义气丶奶', '无敌铁锅', '白菜飞', 'Cris', '枇杷树', '亚太首席技师'], ['点恋线', 'Hoyt', '2. 小水果刀 单挂杯子', '冰封de记忆']]
[['奶', '拳', '火', '近战', '近战', '近战'], ['奶', '眼', '远程', '远程', '眼', '眼'], ['奶', '近战', '近战', '近战', '眼', '眼'], ['眼', '眼', '', '冰雷']]
Warnings: ['1 IDs have empty/unknown job_type.', 'Team4缺奶']
Errors: []
Unmapped: ['2. 小水果刀 单挂杯子']
Invalid: []
Leftover: {}


In [44]:
from openpyxl import load_workbook

# 1) 载入模板
wb = load_workbook("一条排班.xlsx")
ws = wb["Sheet1"]  # 替换为你的工作表名

In [45]:
from datetime import datetime
now = datetime.now()
alphbet = [["A", "B"], ["C", "D"], ["E", "F"], ["G", "H"]]

for i in range(6):
    for t_i, (id_i, job_i) in enumerate(zip(report.grouped, report.grouped_jobs)):
        if i >= len(id_i):
            ws[f'{alphbet[t_i][0]}{i+2}'].value = ""
            ws[f'{alphbet[t_i][1]}{i+2}'].value = ""
        else:
            ws[f'{alphbet[t_i][0]}{i+2}'].value = id_i[i]
            ws[f'{alphbet[t_i][1]}{i+2}'].value = job_i[i]

wb.save(f"{now.strftime('%Y%m%d')}一条.xlsx")

In [ ]:
from tabulate import tabulate
from IPython.display import Markdown

data = []
for i in range(6):
    data_i = []
    for t_i, (id_i, job_i) in enumerate(zip(report.grouped, report.grouped_jobs)):
        if i >= len(id_i):
            data_i.extend(["",""])
        else:
            data_i.extend([id_i[i], job_i[i]])
    data.append(data_i)

headers = ["ID","职业"] * len(report.grouped)
md = tabulate(data, headers=headers, tablefmt="github")
display(Markdown(md))